In [26]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [27]:
ROOT_PATH = Path(".").resolve().parents[0] # please change here
INPUT_PATH = ROOT_PATH / "input" / "bengaliai-cv19" / "train_images"
train_df = pd.read_csv(ROOT_PATH / "input" / "bengaliai-cv19" / "train.csv")

In [37]:
le = LabelEncoder()
le = le.fit(train_df['grapheme'])
train_df['char'] = le.transform(train_df['grapheme'])

In [73]:
train_df['unique_label'] = train_df["grapheme_root"] * 1 + train_df["vowel_diacritic"] * 1000 + train_df["consonant_diacritic"] * 100000
for i in train_df["unique_label"].value_counts().keys():
    tmp_df = train_df.query(f"unique_label=={i}")
    if len(tmp_df["grapheme"].value_counts().keys()) > 1:
        print(tmp_df["grapheme"].value_counts())
        print(tmp_df["grapheme_root"].value_counts())
        print(tmp_df["vowel_diacritic"].value_counts())
        print(tmp_df["consonant_diacritic"].value_counts())
        print(len(tmp_df) / len(train_df) * 100)
        print("===========================================")

র্তে      153
র্ত্রে    150
Name: grapheme, dtype: int64
64    303
Name: grapheme_root, dtype: int64
7    303
Name: vowel_diacritic, dtype: int64
2    303
Name: consonant_diacritic, dtype: int64
0.15086636128261302
র্দ্র    151
র্দ      146
Name: grapheme, dtype: int64
72    297
Name: grapheme_root, dtype: int64
0    297
Name: vowel_diacritic, dtype: int64
2    297
Name: consonant_diacritic, dtype: int64
0.14787890858394742
র্ত্রী    145
র্তী      144
Name: grapheme, dtype: int64
64    289
Name: grapheme_root, dtype: int64
3    289
Name: vowel_diacritic, dtype: int64
2    289
Name: consonant_diacritic, dtype: int64
0.14389563831905994


## vowel

In [110]:
for i in train_df["vowel_diacritic"].value_counts().keys():
    print("vowel: ",  i)
    v = train_df.query(f"vowel_diacritic=={i} and consonant_diacritic!=0")
    for a in v.head(2).values:
        print(a, list(a[4]))
    for a in v.tail(2).values:
        print(a, list(a[4]))

vowel:  0
['Train_11' 107 0 3 'র্ব্য' 300107 959] ['র', '্', 'ব', '্', 'য']
['Train_20' 42 0 2 'র্ছ' 200042 876] ['র', '্', 'ছ']
['Train_200806' 147 0 5 'স্র' 500147 1241] ['স', '্', 'র']
['Train_200821' 159 0 4 'হ্য' 400159 1269] ['হ', '্', 'য']
vowel:  1
['Train_10' 115 1 2 'র্মা' 201115 968] ['র', '্', 'ম', 'া']
['Train_12' 74 1 2 'র্দ্দা' 201074 926] ['র', '্', 'দ', '্', 'দ', 'া']
['Train_200834' 144 1 5 'ষ্প্রা' 501144 1136] ['ষ', '্', 'প', '্', 'র', 'া']
['Train_200837' 2 1 4 'অ্যা' 401002 3] ['অ', '্', 'য', 'া']
vowel:  7
['Train_9' 64 7 1 'তেঁ' 107064 378] ['ত', 'ে', 'ঁ']
['Train_95' 122 7 4 'য্যে' 407122 837] ['য', '্', 'য', 'ে']
['Train_200817' 64 7 5 'ত্রে' 507064 422] ['ত', '্', 'র', 'ে']
['Train_200835' 22 7 2 'র্খে' 207022 860] ['র', '্', 'খ', 'ে']
vowel:  2
['Train_3' 53 2 2 'র্টি' 202053 888] ['র', '্', 'ট', 'ি']
['Train_15' 72 2 2 'র্দি' 202072 919] ['র', '্', 'দ', 'ি']
['Train_200778' 159 2 4 'হ্যি' 402159 1271] ['হ', '্', 'য', 'ি']
['Train_200818' 107 2 5 'ব্রি' 5021

In [105]:
for i in train_df["vowel_diacritic"].value_counts().keys():
    print("vowel: ",  i)
    v = train_df.query(f"vowel_diacritic=={i} and consonant_diacritic==0")
    for a in v.head(2).values:
        print(a, list(a[4]))
    for a in v.tail(2).values:
        print(a, list(a[4]))

vowel:  0
['Train_1' 159 0 0 'হ' 159 1248] ['হ']
['Train_8' 67 0 0 'ত্থ' 67 394] ['ত', '্', 'থ']
['Train_200802' 137 0 0 'শ্ম' 137 1075] ['শ', '্', 'ম']
['Train_200833' 130 0 0 'ল্ব' 130 1038] ['ল', '্', 'ব']
vowel:  1
['Train_13' 100 1 0 'প্পা' 1100 655] ['প', '্', 'প', 'া']
['Train_25' 61 1 0 'ণ্ঠা' 1061 356] ['ণ', '্', 'ঠ', 'া']
['Train_200810' 143 1 0 'ষ্ণা' 1143 1130] ['ষ', '্', 'ণ', 'া']
['Train_200815' 13 1 0 'কা' 1013 20] ['ক', 'া']
vowel:  7
['Train_19' 23 7 0 'গে' 7023 113] ['গ', 'ে']
['Train_37' 120 7 0 'ম্মে' 7120 815] ['ম', '্', 'ম', 'ে']
['Train_200797' 61 7 0 'ণ্ঠে' 7061 358] ['ণ', '্', 'ঠ', 'ে']
['Train_200827' 149 7 0 'স্টে' 7149 1173] ['স', '্', 'ট', 'ে']
vowel:  2
['Train_6' 52 2 0 'ঞ্জি' 2052 282] ['ঞ', '্', 'জ', 'ি']
['Train_34' 52 2 0 'ঞ্জি' 2052 282] ['ঞ', '্', 'জ', 'ি']
['Train_200830' 39 2 0 'চ্চি' 2039 207] ['চ', '্', 'চ', 'ি']
['Train_200839' 127 2 0 'ল্টি' 2127 1023] ['ল', '্', 'ট', 'ি']
vowel:  4
['Train_14' 48 4 0 'ঝু' 4048 267] ['ঝ', 'ু']
['Train_21' 60 4

## Check the existence

### vowel

In [119]:
for n in range(11):
    g = []
    for v in range(168):
        if v not in sorted(train_df.query(f"vowel_diacritic=={n}")["grapheme_root"].value_counts().keys()):
            g.append(v)
    print(n, ": ", g)

0 :  [26, 28, 33, 34, 73, 82, 108, 114, 126, 152, 157, 158, 163]
1 :  [0, 1, 3, 4, 5, 6, 7, 8, 10, 11, 12, 19, 20, 26, 27, 30, 33, 37, 45, 51, 63, 68, 80, 82, 84, 87, 90, 93, 102, 104, 105, 108, 110, 126, 130, 131, 137, 138, 145, 146, 154, 158, 160, 164, 166]
2 :  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 19, 20, 24, 26, 41, 45, 46, 49, 60, 63, 67, 73, 78, 80, 82, 87, 97, 99, 100, 102, 104, 105, 106, 114, 116, 121, 126, 130, 135, 143, 144, 145, 146, 154, 157, 158, 161, 162, 163, 164, 166]
3 :  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 19, 24, 26, 27, 28, 29, 30, 31, 33, 34, 35, 37, 39, 40, 41, 42, 45, 47, 48, 49, 50, 51, 54, 58, 60, 61, 63, 67, 73, 75, 77, 78, 80, 83, 84, 87, 94, 97, 98, 99, 101, 102, 104, 105, 106, 108, 110, 111, 112, 114, 121, 126, 127, 128, 130, 131, 134, 135, 136, 143, 144, 145, 146, 148, 151, 152, 157, 158, 160, 161, 163, 164, 166]
4 :  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 19, 20, 21, 24, 26, 27, 30, 33, 34, 35, 37, 41, 45, 

In [120]:
for n in range(11):
    g = []
    for v in range(168):
        if v in sorted(train_df.query(f"vowel_diacritic=={n}")["grapheme_root"].value_counts().keys()):
            g.append(v)
    print(n, ": ", g)

0 :  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 29, 30, 31, 32, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 74, 75, 76, 77, 78, 79, 80, 81, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 109, 110, 111, 112, 113, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 153, 154, 155, 156, 159, 160, 161, 162, 164, 165, 166, 167]
1 :  [2, 9, 13, 14, 15, 16, 17, 18, 21, 22, 23, 24, 25, 28, 29, 31, 32, 34, 35, 36, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 64, 65, 66, 67, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 81, 83, 85, 86, 88, 89, 91, 92, 94, 95, 96, 97, 98, 99, 100, 101, 103, 106, 107, 109, 1

### consonant

In [137]:
for n in range(8):
    g = []
    for v in range(168):
        if v in sorted(train_df.query(f"consonant_diacritic=={n}")["grapheme_root"].value_counts().keys()):
            g.append(v)
    print(n, ": ", g)

0 :  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167]
1 :  [3, 4, 6, 9, 13, 22, 23, 29, 38, 42, 48, 55, 64, 72, 79, 96, 103, 107, 113, 123, 147, 159]
2 :  [13, 21, 22, 23, 29, 38, 42, 43, 48, 53, 56, 59, 64, 65, 71, 72, 74, 75, 76, 79, 80, 81, 86, 96, 103, 107, 111, 113, 115, 120, 122, 124, 128, 133, 136, 139, 147, 1

In [138]:
for n in range(8):
    g = []
    for v in range(168):
        if v not in sorted(train_df.query(f"consonant_diacritic=={n}")["grapheme_root"].value_counts().keys()):
            g.append(v)
    print(n, ": ", g)

0 :  []
1 :  [0, 1, 2, 5, 7, 8, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 56, 57, 58, 59, 60, 61, 62, 63, 65, 66, 67, 68, 69, 70, 71, 73, 74, 75, 76, 77, 78, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 97, 98, 99, 100, 101, 102, 104, 105, 106, 108, 109, 110, 111, 112, 114, 115, 116, 117, 118, 119, 120, 121, 122, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 160, 161, 162, 163, 164, 165, 166, 167]
2 :  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 44, 45, 46, 47, 49, 50, 51, 52, 54, 55, 57, 58, 60, 61, 62, 63, 66, 67, 68, 69, 70, 73, 77, 78, 82, 83, 84, 85, 87, 88, 89, 90, 91, 92, 93, 94, 95, 97, 98, 99, 100, 101, 102, 104, 105, 106, 108, 109, 110, 1

In [141]:
for n in range(8):
    g = []
    for v in range(11):
        if v in sorted(train_df.query(f"consonant_diacritic=={n}")["vowel_diacritic"].value_counts().keys()):
            g.append(v)
    print(n, ": ", g)

0 :  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
1 :  [0, 1, 2, 4, 7, 9, 10]
2 :  [0, 1, 2, 3, 4, 5, 6, 7, 9]
3 :  [0]
4 :  [0, 1, 2, 4, 5, 7, 8, 9]
5 :  [0, 1, 2, 3, 4, 5, 7, 8, 9, 10]
6 :  [0, 1]
7 :  []


In [142]:
for n in range(8):
    g = []
    for v in range(11):
        if v not in sorted(train_df.query(f"consonant_diacritic=={n}")["vowel_diacritic"].value_counts().keys()):
            g.append(v)
    print(n, ": ", g)

0 :  []
1 :  [3, 5, 6, 8]
2 :  [8, 10]
3 :  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
4 :  [3, 6, 10]
5 :  [6]
6 :  [2, 3, 4, 5, 6, 7, 8, 9, 10]
7 :  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [143]:
for n in range(168):
    g = []
    for v in range(11):
        if v in sorted(train_df.query(f"grapheme_root=={n}")["vowel_diacritic"].value_counts().keys()):
            g.append(v)
    print(n, ": ", g)

0 :  [0]
1 :  [0]
2 :  [0, 1]
3 :  [0]
4 :  [0]
5 :  [0]
6 :  [0]
7 :  [0]
8 :  [0]
9 :  [0, 1]
10 :  [0]
11 :  [0]
12 :  [0]
13 :  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
14 :  [0, 1, 2, 7, 9]
15 :  [0, 1, 2, 7, 9]
16 :  [0, 1, 2, 6, 7, 9]
17 :  [0, 1, 2, 7, 9]
18 :  [0, 1, 2, 3, 4, 7, 9, 10]
19 :  [0, 10]
20 :  [0, 3]
21 :  [0, 1, 2, 3, 7]
22 :  [0, 1, 2, 3, 4, 6, 7, 8, 9]
23 :  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
24 :  [0, 1]
25 :  [0, 1, 2, 3, 4, 7]
26 :  [7]
27 :  [0, 2]
28 :  [1, 2, 4, 9]
29 :  [0, 1, 2, 4, 5, 6, 7, 9]
30 :  [0, 2, 7]
31 :  [0, 1, 2, 4, 7]
32 :  [0, 1, 2, 3, 4, 6, 7, 9]
33 :  [2]
34 :  [1, 2]
35 :  [0, 1, 2]
36 :  [0, 1, 2, 3, 4, 7, 9]
37 :  [0, 2]
38 :  [0, 1, 2, 3, 4, 5, 7, 8, 9, 10]
39 :  [0, 1, 2, 4]
40 :  [0, 1, 2, 4, 7]
41 :  [0, 1]
42 :  [0, 1, 2, 4, 7, 9]
43 :  [0, 1, 2, 3, 4, 7, 8, 9]
44 :  [0, 1, 2, 3, 4, 7, 9]
45 :  [0]
46 :  [0, 1, 3, 7]
47 :  [0, 1, 2]
48 :  [0, 1, 2, 4, 7, 9]
49 :  [0, 1]
50 :  [0, 1, 2, 4, 7]
51 :  [0, 2]
52 :  [0, 1, 2, 3, 4, 7]
53 :  [

In [144]:
for n in range(168):
    g = []
    for v in range(11):
        if v not in sorted(train_df.query(f"grapheme_root=={n}")["vowel_diacritic"].value_counts().keys()):
            g.append(v)
    print(n, ": ", g)

0 :  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
1 :  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
2 :  [2, 3, 4, 5, 6, 7, 8, 9, 10]
3 :  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
4 :  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
5 :  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
6 :  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
7 :  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
8 :  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
9 :  [2, 3, 4, 5, 6, 7, 8, 9, 10]
10 :  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
11 :  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
12 :  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
13 :  []
14 :  [3, 4, 5, 6, 8, 10]
15 :  [3, 4, 5, 6, 8, 10]
16 :  [3, 4, 5, 8, 10]
17 :  [3, 4, 5, 6, 8, 10]
18 :  [5, 6, 8]
19 :  [1, 2, 3, 4, 5, 6, 7, 8, 9]
20 :  [1, 2, 4, 5, 6, 7, 8, 9, 10]
21 :  [4, 5, 6, 8, 9, 10]
22 :  [5, 10]
23 :  []
24 :  [2, 3, 4, 5, 6, 7, 8, 9, 10]
25 :  [5, 6, 8, 9, 10]
26 :  [0, 1, 2, 3, 4, 5, 6, 8, 9, 10]
27 :  [1, 3, 4, 5, 6, 7, 8, 9, 10]
28 :  [0, 3, 5, 6, 7, 8, 10]
29 :  [3, 8, 10]
30 :  [1, 3, 4, 5, 6, 8, 9, 10]
31 :  [3, 5, 6, 8, 9, 10]
32 :  [5, 8, 10]
33 :  [0, 1, 3, 4, 5, 6, 7

In [145]:
for n in range(168):
    g = []
    for v in range(7):
        if v in sorted(train_df.query(f"grapheme_root=={n}")["consonant_diacritic"].value_counts().keys()):
            g.append(v)
    print(n, ": ", g)

0 :  [0]
1 :  [0]
2 :  [0, 4]
3 :  [0, 1]
4 :  [0, 1]
5 :  [0]
6 :  [0, 1]
7 :  [0]
8 :  [0]
9 :  [0, 1, 4]
10 :  [0]
11 :  [0]
12 :  [0]
13 :  [0, 1, 2, 4, 5, 6]
14 :  [0]
15 :  [0, 5]
16 :  [0]
17 :  [0]
18 :  [0, 4]
19 :  [0]
20 :  [0]
21 :  [0, 2]
22 :  [0, 1, 2, 4, 5]
23 :  [0, 1, 2, 4, 5, 6]
24 :  [0]
25 :  [0, 4]
26 :  [0]
27 :  [0]
28 :  [0, 4]
29 :  [0, 1, 2, 3, 4, 5]
30 :  [0]
31 :  [0]
32 :  [0]
33 :  [0]
34 :  [0]
35 :  [0]
36 :  [0]
37 :  [0]
38 :  [0, 1, 2, 4]
39 :  [0]
40 :  [0]
41 :  [0]
42 :  [0, 1, 2, 4]
43 :  [0, 2, 3, 4, 5]
44 :  [0]
45 :  [0]
46 :  [0]
47 :  [0]
48 :  [0, 1, 2]
49 :  [0]
50 :  [0]
51 :  [0]
52 :  [0]
53 :  [0, 2, 4, 5, 6]
54 :  [0]
55 :  [0, 1, 4]
56 :  [0, 2, 4, 5]
57 :  [0]
58 :  [0, 4]
59 :  [0, 2, 4]
60 :  [0]
61 :  [0]
62 :  [0]
63 :  [0]
64 :  [0, 1, 2, 4, 5, 6]
65 :  [0, 2, 4]
66 :  [0]
67 :  [0]
68 :  [0]
69 :  [0]
70 :  [0, 4]
71 :  [0, 2, 3, 4, 5]
72 :  [0, 1, 2, 4, 5, 6]
73 :  [0]
74 :  [0, 2]
75 :  [0, 2]
76 :  [0, 2, 4]
77 :  [0, 5]
78

In [146]:
for n in range(168):
    g = []
    for v in range(7):
        if v not in sorted(train_df.query(f"grapheme_root=={n}")["consonant_diacritic"].value_counts().keys()):
            g.append(v)
    print(n, ": ", g)

0 :  [1, 2, 3, 4, 5, 6]
1 :  [1, 2, 3, 4, 5, 6]
2 :  [1, 2, 3, 5, 6]
3 :  [2, 3, 4, 5, 6]
4 :  [2, 3, 4, 5, 6]
5 :  [1, 2, 3, 4, 5, 6]
6 :  [2, 3, 4, 5, 6]
7 :  [1, 2, 3, 4, 5, 6]
8 :  [1, 2, 3, 4, 5, 6]
9 :  [2, 3, 5, 6]
10 :  [1, 2, 3, 4, 5, 6]
11 :  [1, 2, 3, 4, 5, 6]
12 :  [1, 2, 3, 4, 5, 6]
13 :  [3]
14 :  [1, 2, 3, 4, 5, 6]
15 :  [1, 2, 3, 4, 6]
16 :  [1, 2, 3, 4, 5, 6]
17 :  [1, 2, 3, 4, 5, 6]
18 :  [1, 2, 3, 5, 6]
19 :  [1, 2, 3, 4, 5, 6]
20 :  [1, 2, 3, 4, 5, 6]
21 :  [1, 3, 4, 5, 6]
22 :  [3, 6]
23 :  [3]
24 :  [1, 2, 3, 4, 5, 6]
25 :  [1, 2, 3, 5, 6]
26 :  [1, 2, 3, 4, 5, 6]
27 :  [1, 2, 3, 4, 5, 6]
28 :  [1, 2, 3, 5, 6]
29 :  [6]
30 :  [1, 2, 3, 4, 5, 6]
31 :  [1, 2, 3, 4, 5, 6]
32 :  [1, 2, 3, 4, 5, 6]
33 :  [1, 2, 3, 4, 5, 6]
34 :  [1, 2, 3, 4, 5, 6]
35 :  [1, 2, 3, 4, 5, 6]
36 :  [1, 2, 3, 4, 5, 6]
37 :  [1, 2, 3, 4, 5, 6]
38 :  [3, 5, 6]
39 :  [1, 2, 3, 4, 5, 6]
40 :  [1, 2, 3, 4, 5, 6]
41 :  [1, 2, 3, 4, 5, 6]
42 :  [3, 5, 6]
43 :  [1, 6]
44 :  [1, 2, 3, 4, 5, 6]
45 : 

## consonant

In [135]:
for i in train_df["consonant_diacritic"].value_counts().keys():
    print("consonant: ",  i)
    v = train_df.query(f"consonant_diacritic=={i} and vowel_diacritic!=0")
    print(len(v))
    for a in v.head(2).values:
        print(a, list(a[4]))
    for a in v.tail(2).values:
        print(a, list(a[4]))

consonant:  0
101318
['Train_5' 153 9 0 'স্পো' 9153 1215] ['স', '্', 'প', 'ো']
['Train_6' 52 2 0 'ঞ্জি' 2052 282] ['ঞ', '্', 'জ', 'ি']
['Train_200838' 152 9 0 'স্নো' 9152 1208] ['স', '্', 'ন', 'ো']
['Train_200839' 127 2 0 'ল্টি' 2127 1023] ['ল', '্', 'ট', 'ি']
consonant:  2
17203
['Train_3' 53 2 2 'র্টি' 202053 888] ['র', '্', 'ট', 'ি']
['Train_10' 115 1 2 'র্মা' 201115 968] ['র', '্', 'ম', 'া']
['Train_200831' 81 9 2 'র্নো' 209081 943] ['র', '্', 'ন', 'ো']
['Train_200835' 22 7 2 'র্খে' 207022 860] ['র', '্', 'খ', 'ে']
consonant:  5
17217
['Train_0' 15 9 5 'ক্ট্রো' 509015 44] ['ক', '্', 'ট', '্', 'র', 'ো']
['Train_2' 22 3 5 'খ্রী' 503022 103] ['খ', '্', 'র', 'ী']
['Train_200818' 107 2 5 'ব্রি' 502107 735] ['ব', '্', 'র', 'ি']
['Train_200834' 144 1 5 'ষ্প্রা' 501144 1136] ['ষ', '্', 'প', '্', 'র', 'া']
consonant:  4
15857
['Train_36' 167 1 4 'য়্যা' 401167 1294] ['য়', '্', 'য', 'া']
['Train_44' 38 5 4 'চ্যূ' 405038 219] ['চ', '্', 'য', 'ূ']
['Train_200822' 38 5 4 'চ্যূ' 405038 219] ['চ',

In [108]:
for i in train_df["consonant_diacritic"].value_counts().keys():
    print("consonant: ",  i)
    v = train_df.query(f"consonant_diacritic=={i} and vowel_diacritic==0")
    for a in v.head(2).values:
        print(a, list(a[4]))
    for a in v.tail(2).values:
        print(a, list(a[4]))

consonant:  0
['Train_1' 159 0 0 'হ' 159 1248] ['হ']
['Train_8' 67 0 0 'ত্থ' 67 394] ['ত', '্', 'থ']
['Train_200802' 137 0 0 'শ্ম' 137 1075] ['শ', '্', 'ম']
['Train_200833' 130 0 0 'ল্ব' 130 1038] ['ল', '্', 'ব']
consonant:  2
['Train_20' 42 0 2 'র্ছ' 200042 876] ['র', '্', 'ছ']
['Train_23' 22 0 2 'র্খ' 200022 858] ['র', '্', 'খ']
['Train_200770' 159 0 2 'র্হ' 200159 1004] ['র', '্', 'হ']
['Train_200799' 53 0 2 'র্ট' 200053 886] ['র', '্', 'ট']
consonant:  5
['Train_43' 23 0 5 'গ্র' 500023 135] ['গ', '্', 'র']
['Train_70' 148 0 5 'স্ক্র' 500148 1165] ['স', '্', 'ক', '্', 'র']
['Train_200671' 13 0 5 'ক্র' 500013 55] ['ক', '্', 'র']
['Train_200806' 147 0 5 'স্র' 500147 1241] ['স', '্', 'র']
consonant:  4
['Train_116' 71 0 4 'থ্য' 400071 434] ['থ', '্', 'য']
['Train_150' 70 0 4 'ত্ম্য' 400070 410] ['ত', '্', 'ম', '্', 'য']
['Train_200748' 43 0 4 'জ্য' 400043 255] ['জ', '্', 'য']
['Train_200821' 159 0 4 'হ্য' 400159 1269] ['হ', '্', 'য']
consonant:  1
['Train_192' 6 0 1 'উঁ' 100006 10] ['উ

## g, v, c = 72, 0, X

In [61]:
train_df.query("grapheme_root==72 and vowel_diacritic==0")["consonant_diacritic"].value_counts()

2    297
4    150
5    149
0    148
6    139
Name: consonant_diacritic, dtype: int64

In [66]:
a = train_df.query("grapheme_root==72 and vowel_diacritic==0")["grapheme"].value_counts()
for k, v in a.items():
    b = list(k)
    print(k, v, b)

র্দ্র 151 ['র', '্', 'দ', '্', 'র']
দ্য 150 ['দ', '্', 'য']
দ্র 149 ['দ', '্', 'র']
দ 148 ['দ']
র্দ 146 ['র', '্', 'দ']
দ্র্য 139 ['দ', '্', 'র', '্', 'য']


## g, v, c = 72, X, 2

In [55]:
train_df.query("grapheme_root==72 and consonant_diacritic==2")["vowel_diacritic"].value_counts()

0    297
2    166
7    165
6    150
4    145
3    144
9    143
1    143
Name: vowel_diacritic, dtype: int64

In [67]:
a = train_df.query("grapheme_root==72 and consonant_diacritic==2")["grapheme"].value_counts()
for k, v in a.items():
    b = list(k)
    print(k, v, b)

র্দি 166 ['র', '্', 'দ', 'ি']
র্দে 165 ['র', '্', 'দ', 'ে']
র্দ্র 151 ['র', '্', 'দ', '্', 'র']
র্দৃ 150 ['র', '্', 'দ', 'ৃ']
র্দ 146 ['র', '্', 'দ']
র্দু 145 ['র', '্', 'দ', 'ু']
র্দী 144 ['র', '্', 'দ', 'ী']
র্দা 143 ['র', '্', 'দ', 'া']
র্দো 143 ['র', '্', 'দ', 'ো']


Which is (72, 0, 2)?

র্দ্র & র্দ -> র্দ

## g, v, c = 64, X, 2

In [48]:
train_df.query("grapheme_root==64 and consonant_diacritic==2")["vowel_diacritic"].value_counts()

7    303
3    289
2    167
4    151
1    150
0    150
6    147
Name: vowel_diacritic, dtype: int64

In [65]:
a = train_df.query("grapheme_root==64 and consonant_diacritic==2")["grapheme"].value_counts()
for k, v in a.items():
    b = list(k)
    print(k, v, b)

র্তি 167 ['র', '্', 'ত', 'ি']
র্তে 153 ['র', '্', 'ত', 'ে']
র্তু 151 ['র', '্', 'ত', 'ু']
র্তা 150 ['র', '্', 'ত', 'া']
র্ত্রে 150 ['র', '্', 'ত', '্', 'র', 'ে']
র্ত 150 ['র', '্', 'ত']
র্তৃ 147 ['র', '্', 'ত', 'ৃ']
র্ত্রী 145 ['র', '্', 'ত', '্', 'র', 'ী']
র্তী 144 ['র', '্', 'ত', 'ী']


## g, v, c = 64, 7, X

In [70]:
train_df.query("grapheme_root==64 and vowel_diacritic==7")["consonant_diacritic"].value_counts()

2    303
4    170
5    166
0    155
1    147
Name: consonant_diacritic, dtype: int64

In [71]:
a = train_df.query("grapheme_root==64 and vowel_diacritic==7")["grapheme"].value_counts()
for k, v in a.items():
    b = list(k)
    print(k, v, b)

ত্যে 170 ['ত', '্', 'য', 'ে']
ত্রে 166 ['ত', '্', 'র', 'ে']
তে 155 ['ত', 'ে']
র্তে 153 ['র', '্', 'ত', 'ে']
র্ত্রে 150 ['র', '্', 'ত', '্', 'র', 'ে']
তেঁ 147 ['ত', 'ে', 'ঁ']


Which is (62, 7, 2)? র্তে, র্ত্রে -> র্ত্রে

In [154]:
a = train_df.query("grapheme_root==64")["grapheme"].value_counts()
for k, v in a.items():
    b = list(k)
    print(k, v, b)

তৃ 175 ['ত', 'ৃ']
ত্রী 174 ['ত', '্', 'র', 'ী']
তু 171 ['ত', 'ু']
ত্যে 170 ['ত', '্', 'য', 'ে']
ত্র 170 ['ত', '্', 'র']
তৌ 169 ['ত', 'ৌ']
ত 169 ['ত']
তি 168 ['ত', 'ি']
ত্র্য 167 ['ত', '্', 'র', '্', 'য']
ত্যা 167 ['ত', '্', 'য', 'া']
র্তি 167 ['র', '্', 'ত', 'ি']
ত্রে 166 ['ত', '্', 'র', 'ে']
ত্রা 164 ['ত', '্', 'র', 'া']
তে 155 ['ত', 'ে']
র্তে 153 ['র', '্', 'ত', 'ে']
ত্রো 153 ['ত', '্', 'র', 'ো']
ত্য 152 ['ত', '্', 'য']
ত্রৈ 151 ['ত', '্', 'র', 'ৈ']
তো 151 ['ত', 'ো']
র্তু 151 ['র', '্', 'ত', 'ু']
র্তা 150 ['র', '্', 'ত', 'া']
র্ত্রে 150 ['র', '্', 'ত', '্', 'র', 'ে']
র্ত 150 ['র', '্', 'ত']
তা 150 ['ত', 'া']
ত্যু 148 ['ত', '্', 'য', 'ু']
তী 148 ['ত', 'ী']
র্তৃ 147 ['র', '্', 'ত', 'ৃ']
তেঁ 147 ['ত', 'ে', 'ঁ']
তূ 146 ['ত', 'ূ']
ত্যি 145 ['ত', '্', 'য', 'ি']
র্ত্রী 145 ['র', '্', 'ত', '্', 'র', 'ী']
তৈ 145 ['ত', 'ৈ']
ত্রি 144 ['ত', '্', 'র', 'ি']
র্তী 144 ['র', '্', 'ত', 'ী']
তাঁ 143 ['ত', 'া', 'ঁ']
ত্রু 131 ['ত', '্', 'র', 'ু']


## g, v, c = 64, 3, X

In [74]:
train_df.query("grapheme_root==64 and vowel_diacritic==3")["consonant_diacritic"].value_counts()

2    289
5    174
0    148
Name: consonant_diacritic, dtype: int64

In [75]:
a = train_df.query("grapheme_root==64 and vowel_diacritic==3")["grapheme"].value_counts()
for k, v in a.items():
    b = list(k)
    print(k, v, b)

ত্রী 174 ['ত', '্', 'র', 'ী']
তী 148 ['ত', 'ী']
র্ত্রী 145 ['র', '্', 'ত', '্', 'র', 'ী']
র্তী 144 ['র', '্', 'ত', 'ী']


Which is 64, 3, X? র্ত্রী, র্তী -> র্তী

In [ ]:
train_df.query("grapheme_root==64 and consonant_diacritic==2")["vowel_diacritic"].value_counts()